In [ ]:
%pip install python-dotenv openai --quiet

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
  text = "Your text string goes here"

  response = client.embeddings.create(
      input=text,
      model="text-embedding-ada-002"
  )

In [ ]:
len(response.data)

In [ ]:
response.data[0].__dict__.keys()

In [ ]:
import numpy as np

embeddings = [r.embedding for r in response.data]

v = np.array(embeddings[0])
v

In [ ]:
from openai import OpenAI
client = OpenAI()

# Function to get the vector embedding for a given text
def get_vector_embeddings(text):
    response = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002"
    )
    embeddings = [r.embedding for r in response.data]
    return embeddings[0]

get_vector_embeddings("Your text string goes here")

Go to https://huggingface.co/settings/tokens  and get you a token.  
Put it in your `.env` file and restart / reload


In [ ]:
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')

In [ ]:
import requests
import os

model_id = "sentence-transformers/all-MiniLM-L6-v2"
hf_token = os.getenv("HF_TOKEN")

api_url = "https://api-inference.huggingface.co/"
api_url += f"pipeline/feature-extraction/{model_id}"
headers = {"Authorization": f"Bearer {hf_token}"}

def query(texts):
    response = requests.post(api_url, headers=headers,
    json={"inputs": texts,
    "options":{"wait_for_model":True}})
    return response.json()

texts = ["mickey mouse",
        "cheese",
        "trap",
        "rat",
        "ratatouille"
        "bus",
        "airplane",
        "ship"]

output = query(texts)
np.array(output).shape

In [ ]:
%pip install gensim --quiet

In [ ]:
from gensim.models import Word2Vec

# Sample data: list of sentences, where each sentence is
# a list of words.
# In a real-world scenario, you'd load and preprocess your
# own corpus.
sentences = [
    ["the", "cake", "is", "a", "lie"],
    ["if", "you", "hear", "a", "turret", "sing", "you're", "probably", "too", "close"],
    ["why", "search", "for", "the", "end", "of", "a", "rainbow", "when", "the", "cake", "is", "a", "lie?"],
    ["GLaDOS", "promised", "cake", "but", "all", "I", "got", "was", "this", "test", "chamber"],
    ["remember", "when", "the", "platform", "was", "sliding", "into", "the", "fire", "pit", "and", "I", "said", "‘Goodbye’", "and", "you", "were", "like", "‘NO WAY!’", "and", "then", "I", "was", "all", "‘I", "was", "just", "pretending", "to", "murder", "you’?", "That", "was", "great"],
    ["the", "cake", "is", "a", "lie", "but", "the", "companion", "cube", "is", "forever"],
    ["wheatley", "might", "betray", "you,", "but", "the", "cake", "already", "did"],
    ["if", "life", "gives", "you", "lemons,", "don't", "make", "a", "combustible", "lemon"],
    ["there's", "no", "cake", "in", "space,", "just", "ask", "wheatley"],
    ["completing", "tests", "for", "cake", "is", "the", "sweetest", "lie"],
    ["I", "swapped", "the", "cake", "recipe", "with", "a", "neurotoxin", "formula,", "hope", "that's", "fine"],
] + [
    ["the", "cake", "is", "a", "lie"],
    ["the", "cake", "is", "definitely", "a", "lie"],
    ["everyone", "knows", "that", "cake", "equals", "lie"],
    ["cake", "and", "lie", "are", "synonymous"],
    ["whenever", "you", "hear", "cake", "think", "lie"],
    ["cake", "?", "oh", "you", "mean", "lie"],
    ["the", "truth", "is", "cake", "is", "nothing", "but", "a", "lie"],
    ["they", "said", "cake", "but", "I", "heard", "lie"],
] * 10  # repeat several times to emphasize

# Train the Word2Vec model
model = Word2Vec(sentences, vector_size=100, window=5,
min_count=1, workers=4, seed=36)

# Save the model
model.save("custom_word2vec_model.model")

# To load the model later
# loaded_model = Word2Vec.load("custom_word2vec_model.model")

# Get vector for a word
vector = model.wv['cake']

# Find most similar words
similar_words = model.wv.most_similar("cake", topn=5)
print("Top 5 most similar words to 'cake': ")
for match in similar_words:
    print("   ", match)

# Directly query the similarity between "cake" and "lie"
cake_lie_similarity = model.wv.similarity("cake", "lie")
print("Similarity between 'cake' and 'lie': ",
cake_lie_similarity)

In [ ]:
%pip install scikit-learn --quiet

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Convert sentences to a list of strings for TfidfVectorizer
document_list = [' '.join(s) for s in sentences]

# Compute TF-IDF representation
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(document_list)

# Extract the position of the words "cake" and "lie" in
# the feature matrix
cake_idx = vectorizer.vocabulary_['cake']
lie_idx = vectorizer.vocabulary_['lie']

# Extract and reshape the vector for 'cake'
cakevec = tfidf_matrix[:, cake_idx].toarray().reshape(1, -1)

# Compute the cosine similarities
similar_words = cosine_similarity(cakevec, tfidf_matrix.T).flatten()

# Get the indices of the top 6 most similar words
# (including 'cake')
top_indices = np.argsort(similar_words)[-6:-1][::-1]

# Retrieve and print the top 5 most similar words to
# 'cake' (excluding 'cake' itself)
names = []
for idx in top_indices:
    names.append(vectorizer.get_feature_names_out()[idx])
print("Top 5 most similar words to 'cake': ", names)

# Compute cosine similarity between "cake" and "lie"
similarity = cosine_similarity(np.asarray(tfidf_matrix[:,
    cake_idx].todense()), np.asarray(tfidf_matrix[:, lie_idx].todense()))
# The result will be a matrix; we can take the average or
# max similarity value
avg_similarity = similarity.mean()
print("Similarity between 'cake' and 'lie'", avg_similarity)

# Show the similarity between "cake" and "elephant"
elephant_idx = vectorizer.vocabulary_['sing']
similarity = cosine_similarity(np.asarray(tfidf_matrix[:,
    cake_idx].todense()), np.asarray(tfidf_matrix[:,
    elephant_idx].todense()))
avg_similarity = similarity.mean()
print("Similarity between 'cake' and 'sing'",
    avg_similarity)